# NOMAD Attribution Module

This notebook provides reusable attribution management UI and logic for NOMAD API access. It can be imported into other dashboard notebooks to help manage attributions for sample uploads.

## Usage

```python
# Import the authentication and data retrieval modules (required)
%run 'nomad_auth.ipynb'
%run 'nomad_data_retrieval.ipynb'

# Import the attribution module
%run 'nomad_attribution.ipynb'

# The following functions are now available:
# - create_attribution_tab(data_state): Creates and returns the attribution management UI
# - create_sample_table(df, editable=False): Creates an interactive table for sample data
```

## Requirements

- The `nomad_auth.ipynb` notebook must be run first for authentication
- The `nomad_data_retrieval.ipynb` notebook must be run for data retrieval functions
- The `nomad_data.py` module should be available for loading and saving attributions

In [ ]:
# Import required libraries
import ipywidgets as widgets
from ipywidgets import HBox, VBox, Button, Text, Label, Dropdown, Checkbox, Output
from IPython.display import display, clear_output, HTML
import pandas as pd
import numpy as np
import json
from datetime import datetime

# Check if dependencies have been imported
try:
    # These variables should be defined in nomad_auth.ipynb
    _ = api_client  # Just to check if it exists
    _ = current_token
    _ = current_user_info
except NameError:
    print("⚠️ Warning: nomad_auth.ipynb must be run before this notebook. Run '%run nomad_auth.ipynb' first.")

try:
    # These functions should be defined in nomad_data_retrieval.ipynb
    _ = get_author_names
    _ = fetch_user_details
except NameError:
    print("⚠️ Warning: nomad_data_retrieval.ipynb must be run before this notebook. Run '%run nomad_data_retrieval.ipynb' first.")

# Import NOMAD data functionality
try:
    from nomad_data import load_attributions, save_attributions
except ImportError:
    print("⚠️ Warning: nomad_data.py module not found. Attribution management functionality may be limited.")
    
    # Provide fallback functionality
    def load_attributions():
        """Fallback function to load attributions from a file."""
        try:
            with open('attribution_overrides.csv', 'r') as f:
                return pd.read_csv(f).set_index('entry_id').to_dict('index')
        except (FileNotFoundError, pd.errors.EmptyDataError):
            return {}
            
    def save_attributions(attributions):
        """Fallback function to save attributions to a file."""
        pd.DataFrame.from_dict(attributions, orient='index').reset_index().rename(columns={'index': 'entry_id'}).to_csv('attribution_overrides.csv', index=False)

## Helper Functions for Attribution Management

In [ ]:
def create_sample_table(df, editable=False):
    """Create an interactive table for sample data.
    
    Args:
        df: DataFrame containing sample data
        editable: Boolean, whether to make the table editable
        
    Returns:
        widgets.Output: Output widget containing the table
    """
    if df is None or df.empty:
        return widgets.HTML("<p>No data available.</p>")
    
    # Columns to display in the table
    display_columns = [
        'sample_id', 'author_name', 'sample_name', 'upload_date',
        'sample_owner', 'comment', 'entry_id'
    ]
    
    # Check which columns are available
    available_columns = [col for col in display_columns if col in df.columns]
    
    # Subset to only available columns
    display_df = df[available_columns].copy()
    
    # Format the date column if it exists
    if 'upload_date' in display_df.columns:
        display_df['upload_date'] = pd.to_datetime(display_df['upload_date']).dt.strftime('%Y-%m-%d')
    
    # Create the output widget
    output = widgets.Output()
    
    with output:
        # Apply custom styling for the table
        display(HTML("""
        <style>
            .sample-table {
                font-family: Arial, sans-serif;
                border-collapse: collapse;
                width: 100%;
                margin-bottom: 20px;
            }
            .sample-table th {
                background-color: #f2f2f2;
                color: #333;
                font-weight: bold;
                text-align: left;
                padding: 8px;
                border: 1px solid #ddd;
            }
            .sample-table td {
                padding: 8px;
                border: 1px solid #ddd;
                text-align: left;
            }
            .sample-table tr:nth-child(even) {
                background-color: #f9f9f9;
            }
            .sample-table tr:hover {
                background-color: #e6f5ff;
            }
        </style>
        """))
        
        # Generate the HTML table
        table_html = ["<table class='sample-table'>"]
        
        # Table header
        table_html.append("<thead><tr>")
        for col in available_columns:
            # Format column headers to be more readable
            header = col.replace('_', ' ').title()
            table_html.append(f"<th>{header}</th>")
        if editable:
            table_html.append("<th>Actions</th>")
        table_html.append("</tr></thead>")
        
        # Table body
        table_html.append("<tbody>")
        for _, row in display_df.iterrows():
            table_html.append("<tr>")
            for col in available_columns:
                value = row[col] if pd.notna(row[col]) else ''
                # Limit long strings
                if isinstance(value, str) and len(value) > 50:
                    value = value[:47] + '...'
                table_html.append(f"<td>{value}</td>")
            if editable:
                entry_id = row['entry_id']
                # Add edit button
                table_html.append(f"<td><button class='edit-btn' data-entry-id='{entry_id}'>Edit</button></td>")
            table_html.append("</tr>")
        table_html.append("</tbody>")
        table_html.append("</table>")
        
        # Display the table
        display(HTML(''.join(table_html)))
        
    return output

## Attribution Tab Component

In [ ]:
def create_attribution_tab(data_state):
    """Create the attribution management tab
    
    Args:
        data_state: Dictionary with data state containing at minimum:
                    - df: DataFrame with sample data
                    - attributions: Dictionary of attribution overrides
                      
    Returns:
        widgets.VBox: Attribution management UI
    """
    # Search input for filtering samples
    search_input = widgets.Text(
        value='',
        placeholder='Search samples by ID, name or author...',
        description='Search:',
        disabled=False,
        layout=widgets.Layout(width='50%')
    )
    
    # Filter button
    filter_button = widgets.Button(
        description='Filter',
        disabled=False,
        button_style='primary',
        tooltip='Apply filter to samples',
        icon='filter'
    )
    
    # Reset filter button
    reset_button = widgets.Button(
        description='Reset',
        disabled=False,
        button_style='',
        tooltip='Reset filter',
        icon='refresh'
    )
    
    # Search bar layout
    search_bar = widgets.HBox([search_input, filter_button, reset_button], 
                              layout=widgets.Layout(width='100%', margin='10px 0px'))
    
    # Attribution editor area
    edit_heading = widgets.HTML("<h3>Edit Sample Attribution</h3>")
    
    # Sample info display
    sample_info = widgets.HTML("<p>Select a sample to edit its attribution.</p>")
    
    # Attribution dropdown
    attribution_dropdown = widgets.Dropdown(
        options=[],
        description='Attributed to:',
        disabled=True,
        layout=widgets.Layout(width='60%')
    )
    
    # Save attribution button
    save_button = widgets.Button(
        description='Save',
        disabled=True,
        button_style='success',
        tooltip='Save attribution changes',
        icon='check'
    )
    
    # Reset attribution button
    revert_button = widgets.Button(
        description='Revert',
        disabled=True,
        button_style='danger',
        tooltip='Revert to original attribution',
        icon='undo'
    )
    
    # Status output
    status_output = widgets.Output()
    
    # Attribution editor layout
    attribution_editor = widgets.VBox([
        edit_heading,
        sample_info,
        attribution_dropdown,
        widgets.HBox([save_button, revert_button]),
        status_output
    ], layout=widgets.Layout(border='1px solid #ddd', padding='10px', margin='10px 0px'))
    
    # Sample table output
    sample_table_output = widgets.Output()
    
    # Function to update the sample table
    def update_sample_table(search_term=None):
        with sample_table_output:
            clear_output()
            
            if data_state['df'] is None or data_state['df'].empty:
                display(widgets.HTML("<p>No data available. Please fetch data first.</p>"))
                return
            
            # Apply search filter if provided
            if search_term and search_term.strip():
                search_term = search_term.lower().strip()
                filtered_df = data_state['df'][data_state['df'].apply(
                    lambda row: any(str(val).lower().find(search_term) >= 0 
                                     for val in row.values if pd.notna(val) and isinstance(val, (str, int, float))),
                    axis=1
                )]
            else:
                filtered_df = data_state['df']
            
            # Display the table
            if filtered_df.empty:
                display(widgets.HTML("<p>No samples match your search criteria.</p>"))
            else:
                display(create_sample_table(filtered_df, editable=True))
                
                # Add JavaScript to handle the edit button clicks
                display(HTML("""
                <script>
                    // Function to handle edit button clicks
                    function setupEditButtons() {
                        const editButtons = document.querySelectorAll('.edit-btn');
                        editButtons.forEach(button => {
                            button.addEventListener('click', function() {
                                const entryId = this.getAttribute('data-entry-id');
                                // Send message to Python
                                IPython.notebook.kernel.execute(
                                    `edit_sample("${entryId}")`
                                );
                            });
                        });
                    }
                    
                    // Wait for the DOM to load completely
                    setTimeout(setupEditButtons, 500);
                </script>
                """))
    
    # Event handlers
    def on_filter_button_clicked(b):
        update_sample_table(search_input.value)
    
    def on_reset_button_clicked(b):
        search_input.value = ''
        update_sample_table()
    
    # Current sample being edited
    current_sample = {'entry_id': None, 'original_author': None}
    
    def edit_sample(entry_id):
        """Function to handle edit button clicks from JavaScript"""
        if data_state['df'] is None or entry_id not in data_state['df']['entry_id'].values:
            return
        
        # Get the sample data
        sample = data_state['df'][data_state['df']['entry_id'] == entry_id].iloc[0]
        
        # Update the sample info display
        sample_info.value = f"""
        <div>
            <p><strong>Sample ID:</strong> {sample.get('sample_id', 'N/A')}</p>
            <p><strong>Sample Name:</strong> {sample.get('sample_name', 'N/A')}</p>
            <p><strong>Current Attribution:</strong> {sample.get('author_name', 'Unknown')}</p>
            <p><strong>Upload Date:</strong> {pd.to_datetime(sample.get('upload_date')).strftime('%Y-%m-%d') if pd.notna(sample.get('upload_date')) else 'N/A'}</p>
        </div>
        """
        
        # Update the current sample
        current_sample['entry_id'] = entry_id
        current_sample['original_author'] = sample.get('main_author')
        
        # Get unique authors for the dropdown
        unique_authors = data_state['df'][['main_author', 'author_name']].drop_duplicates()
        unique_authors = unique_authors[unique_authors['main_author'].notna()]
        
        # Add custom option for "no attribution"
        author_options = [("No Attribution", None)]
        
        # Add existing authors
        for _, row in unique_authors.iterrows():
            author_id = row['main_author']
            author_name = row['author_name']
            if pd.notna(author_id) and author_id:
                author_options.append((author_name, author_id))
        
        # Update attribution dropdown
        attribution_dropdown.options = author_options
        
        # Set current value from attributions or original
        current_attribution = None
        if entry_id in data_state['attributions']:
            current_attribution = data_state['attributions'][entry_id].get('main_author')
        else:
            current_attribution = sample.get('main_author')
        
        # Find the option that matches current_attribution
        for name, value in author_options:
            if value == current_attribution:
                attribution_dropdown.value = value
                break
        else:
            # Default to first option if no match found
            if author_options:
                attribution_dropdown.value = author_options[0][1]
        
        # Enable the dropdown and buttons
        attribution_dropdown.disabled = False
        save_button.disabled = False
        revert_button.disabled = False
        
    # Make edit_sample accessible to JavaScript
    import IPython
    IPython.display.display(IPython.core.display.Javascript("""
    window.edit_sample = function(entry_id) {
        IPython.notebook.kernel.execute(`edit_sample("${entry_id}")`)
    }
    """))
    
    # Event handlers for save and revert buttons
    def on_save_button_clicked(b):
        entry_id = current_sample['entry_id']
        if entry_id is None:
            return
        
        with status_output:
            clear_output()
            print(f"Saving attribution changes for sample {entry_id}...")
            
            try:
                # Check if this is a change
                new_author = attribution_dropdown.value
                
                # If reverting to original, remove from attributions
                if new_author == current_sample['original_author']:
                    if entry_id in data_state['attributions']:
                        del data_state['attributions'][entry_id]
                else:
                    # Add to attributions
                    if entry_id not in data_state['attributions']:
                        data_state['attributions'][entry_id] = {}
                    
                    data_state['attributions'][entry_id]['main_author'] = new_author
                    data_state['attributions'][entry_id]['override_date'] = datetime.now().isoformat()
                    data_state['attributions'][entry_id]['override_by'] = current_user_info.get('username', 'unknown') if current_user_info else 'unknown'
                
                # Save attributions to file
                save_attributions(data_state['attributions'])
                
                # Update the sample in the dataframe
                idx = data_state['df'].index[data_state['df']['entry_id'] == entry_id].tolist()
                if idx:
                    data_state['df'].at[idx[0], 'main_author'] = new_author
                    
                    # Update author_name
                    if new_author is None or pd.isna(new_author):
                        data_state['df'].at[idx[0], 'author_name'] = 'No Attribution'
                    else:
                        # Find the author name from attribution_dropdown.options
                        for name, value in attribution_dropdown.options:
                            if value == new_author:
                                data_state['df'].at[idx[0], 'author_name'] = name
                                break
                
                print(f"✓ Attribution saved successfully!")
                
                # Update the sample info display
                sample = data_state['df'][data_state['df']['entry_id'] == entry_id].iloc[0]
                sample_info.value = f"""
                <div>
                    <p><strong>Sample ID:</strong> {sample.get('sample_id', 'N/A')}</p>
                    <p><strong>Sample Name:</strong> {sample.get('sample_name', 'N/A')}</p>
                    <p><strong>Current Attribution:</strong> {sample.get('author_name', 'Unknown')}</p>
                    <p><strong>Upload Date:</strong> {pd.to_datetime(sample.get('upload_date')).strftime('%Y-%m-%d') if pd.notna(sample.get('upload_date')) else 'N/A'}</p>
                </div>
                """
                
                # Refresh the sample table
                update_sample_table(search_input.value)
                
            except Exception as e:
                print(f"❌ Error saving attribution: {str(e)}")
                import traceback
                traceback.print_exc()
    
    def on_revert_button_clicked(b):
        entry_id = current_sample['entry_id']
        if entry_id is None:
            return
        
        with status_output:
            clear_output()
            print(f"Reverting attribution for sample {entry_id} to original...")
            
            try:
                # Remove from attributions
                if entry_id in data_state['attributions']:
                    del data_state['attributions'][entry_id]
                
                # Save attributions to file
                save_attributions(data_state['attributions'])
                
                # Update the sample in the dataframe
                idx = data_state['df'].index[data_state['df']['entry_id'] == entry_id].tolist()
                if idx:
                    data_state['df'].at[idx[0], 'main_author'] = current_sample['original_author']
                    
                    # Update author_name
                    if current_sample['original_author'] is None or pd.isna(current_sample['original_author']):
                        data_state['df'].at[idx[0], 'author_name'] = 'No Attribution'
                    else:
                        # Find the author name from attribution_dropdown.options
                        for name, value in attribution_dropdown.options:
                            if value == current_sample['original_author']:
                                data_state['df'].at[idx[0], 'author_name'] = name
                                break
                
                # Update attribution dropdown
                for name, value in attribution_dropdown.options:
                    if value == current_sample['original_author']:
                        attribution_dropdown.value = value
                        break
                
                print(f"✓ Attribution reverted successfully!")
                
                # Update the sample info display
                sample = data_state['df'][data_state['df']['entry_id'] == entry_id].iloc[0]
                sample_info.value = f"""
                <div>
                    <p><strong>Sample ID:</strong> {sample.get('sample_id', 'N/A')}</p>
                    <p><strong>Sample Name:</strong> {sample.get('sample_name', 'N/A')}</p>
                    <p><strong>Current Attribution:</strong> {sample.get('author_name', 'Unknown')}</p>
                    <p><strong>Upload Date:</strong> {pd.to_datetime(sample.get('upload_date')).strftime('%Y-%m-%d') if pd.notna(sample.get('upload_date')) else 'N/A'}</p>
                </div>
                """
                
                # Refresh the sample table
                update_sample_table(search_input.value)
                
            except Exception as e:
                print(f"❌ Error reverting attribution: {str(e)}")
                import traceback
                traceback.print_exc()
    
    # Connect handlers to buttons
    filter_button.on_click(on_filter_button_clicked)
    reset_button.on_click(on_reset_button_clicked)
    save_button.on_click(on_save_button_clicked)
    revert_button.on_click(on_revert_button_clicked)
    
    # Handle Enter key in search input
    def on_search_input_submit(sender):
        update_sample_table(search_input.value)
    
    search_input.on_submit(on_search_input_submit)
    
    # Create the attribution tab
    attribution_tab = widgets.VBox([
        widgets.HTML("<h2>Attribution Management</h2>"),
        widgets.HTML("<p>View and edit attributions for HySprint samples.</p>"),
        search_bar,
        sample_table_output,
        attribution_editor
    ])
    
    # Initial update of the sample table
    update_sample_table()
    
    return attribution_tab

## Direct UI Preview

If you want to see this component directly, you can run the cell below to create a preview with test data.

In [ ]:
# Uncomment and run this cell to see a preview of the attribution management UI
# Note: This creates mock data for demonstration purposes

'''
# Create some test data for preview
import pandas as pd
import random
from datetime import datetime, timedelta

# Mock sample data
def create_mock_data(count=100):
    authors = [f"Author {i}" for i in range(1, 6)]
    data = []
    
    for i in range(count):
        random_days = random.randint(0, 365)
        data.append({
            'upload_id': f"upload_{i:03d}",
            'upload_name': f"Sample Test {i:03d}",
            'main_author': random.choice(authors),
            'author_name': random.choice(authors),
            'upload_date': (datetime.now() - timedelta(days=random_days)).strftime('%Y-%m-%d')
        })
    return pd.DataFrame(data)

# Create mock data state
mock_data_state = {
    'df': create_mock_data(100),
    'attributions': {}  # Start with empty attributions
}

# Create the attribution UI
attribution_ui = create_attribution_tab(mock_data_state)
display(attribution_ui)
'''